In [48]:
import pandas as pd

# Replace 'your_file.csv' with the actual path to your file
file_path = 'dataset\got_all_scripts\Game_of_Thrones_Script.csv'
data = pd.read_csv(file_path)

# Preview the first few rows of your dataset
print(data.head())

  Release Date    Season    Episode     Episode Title          Name  \
0   2011-04-17  Season 1  Episode 1  Winter is Coming  waymar royce   
1   2011-04-17  Season 1  Episode 1  Winter is Coming          will   
2   2011-04-17  Season 1  Episode 1  Winter is Coming  waymar royce   
3   2011-04-17  Season 1  Episode 1  Winter is Coming          will   
4   2011-04-17  Season 1  Episode 1  Winter is Coming         gared   

                                            Sentence  
0  What do you expect? They're savages. One lot s...  
1  I've never seen wildlings do a thing like this...  
2                             How close did you get?  
3                            Close as any man would.  
4                   We should head back to the wall.  


In [49]:
df = data[['Name', 'Sentence']]

In [50]:
# Group by 'Name' and aggregate 'Sentence' into a list
df = df.groupby('Name')['Sentence'].agg(list).reset_index()

# Count the number of sentences for each name
df['Sentence_Count'] = df['Sentence'].apply(len)

# Sort by 'Sentence_Count' in descending order
df = df.sort_values(by='Sentence_Count', ascending=False)

# Get the top 50 names
df = df[df['Name'] != 'man']
df = df[df['Name'] != 'soldier']

In [51]:
df.head(50)

,Name,Sentence,Sentence_Count
509,tyrion lannister,[Mmh. It is true what they say about the North...,1760
193,jon snow,"[Go on. Father's watching., And your mother., ...",1133
86,daenerys targaryen,[We've been his guests for over a year and he'...,1048
70,cersei lannister,[And you never worry about anything. When we w...,1005
180,jaime lannister,"[As your brother, I feel it's my duty to warn ...",945
437,sansa stark,"[Thank you., Will you shut up?, Would you plea...",784
23,arya stark,"[Move!, Where's the Imp?, Arya., That's Jaime ...",783
90,davos,"[Not now., Stannis is our king. We follow wher...",528
492,theon greyjoy,"[Mountain lion?, It's a freak., Right. Give it...",455
354,petyr baelish,[But we have a Kingdom to look after. I've hop...,449


In [86]:
from transformers import pipeline
from collections import Counter
import torch
import pandas as pd
from scipy.spatial.distance import cosine
import json

# Section-1 Create the personality models and find the scores for the characters

# Check if GPU is available
device = 0 if torch.cuda.is_available() else -1

# Initialize Hugging Face pipelines with the appropriate device
sentiment_pipeline = pipeline("sentiment-analysis", device=device)
zero_shot_pipeline = pipeline(model="facebook/bart-large-mnli", device=device)

# Trait keywords for loyalty
trait_keywords = {
    "loyalty": ["family", "honor", "oath", "duty", "friendship", "trust"]
}

# Labels for zero-shot classification
zero_shot_labels = {
    "manipulativeness": ["deception", "manipulation", "power play", "trickery"],
    "ambition": ["ambition", "goal-setting", "career", "dreams"],
    "loyalty": ["faithfulness", "dedication", "allegiance", "trustworthiness", "devotion", "commitment"],
    "humour": ["sarcasm", "joke", "funny", "laugh", "hilarious"],
    "courage": ["bravery", "valor", "fearlessness", "heroism", "boldness", "daring"]
}

# Labels for zero-shot classification
#manipulative_labels = ["deception", "manipulation", "power play", "trickery"]
#ambitious_labels = ["ambition", "power", "goal-setting", "dreams"]

# Trait extraction functions
def extract_courage(dialogues):
    """Extract courage based on positive sentiment."""
    scores = []
    for dialogue in dialogues:
        result = sentiment_pipeline(dialogue)
        if result[0]['label'] == "POSITIVE":
            scores.append(result[0]['score'])
    return sum(scores) / len(scores) if scores else 0

def extract_loyalty(dialogues):
    """Extract loyalty based on keyword matches."""
    total_words = sum(len(dialogue.split()) for dialogue in dialogues)
    keyword_count = sum(
        sum(1 for word in dialogue.split() if word.lower() in trait_keywords["loyalty"])
        for dialogue in dialogues
    )
    return keyword_count / total_words if total_words > 0 else 0

def extract_zeroshot_trait(character, dialogues, labels):
    """Extract traits using zero-shot classification."""
    print(character)
    scores = []
    result = zero_shot_pipeline(" ".join(dialogues), candidate_labels=labels)
    #for dialogue in dialogues:
    #    result = zero_shot_pipeline(dialogue, candidate_labels=labels)
    #    scores.append(max(result["scores"]))
    #scores.append(max(result["scores"]))
    print(max(result["scores"]))
    return max(result["scores"])
    #print(labels)
    #print(sum(scores) / len(scores) if scores else 0)
    #return sum(scores) / len(scores) if scores else 0
    #print(scores)

def calculate_character_traits(character, dialogues):
    """Calculate all traits for a character."""
    return {
        "courage": extract_zeroshot_trait(character, dialogues, zero_shot_labels["courage"]),
        "loyalty": extract_zeroshot_trait(character, dialogues, zero_shot_labels["loyalty"]),
        "manipulativeness": extract_zeroshot_trait(character, dialogues, zero_shot_labels["manipulativeness"]),
        "ambition": extract_zeroshot_trait(character, dialogues, zero_shot_labels["ambition"]),
        "humour": extract_zeroshot_trait(character, dialogues, zero_shot_labels["humour"])
    }

def normalize_traits(character_profiles):
    """Normalize trait scores for consistency."""
    all_traits = {trait: [] for trait in next(iter(character_profiles.values()))}

    # Collect all values for each trait
    for traits in character_profiles.values():
        for trait, value in traits.items():
            all_traits[trait].append(value)

    # Normalize each trait
    normalized_profiles = {}
    for character, traits in character_profiles.items():
        normalized_profiles[character] = {
            trait: (value - min(all_traits[trait])) / (max(all_traits[trait]) - min(all_traits[trait]) + 1e-6)
            for trait, value in traits.items()
        }
    return normalized_profiles


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0
Device set to use cuda:0


In [64]:
# dataset
file_path = 'dataset\got_all_scripts\got_data_cleaned.csv'
dataset = pd.read_csv(file_path)


In [87]:
dataset

,Name,Sentence,Sentence_Count
0,tyrion lannister,['Mmh. It is true what they say about the Nort...,1760
1,jon snow,"[""Go on. Father's watching."", 'And your mother...",1133
2,daenerys targaryen,"[""We've been his guests for over a year and he...",1048
3,cersei lannister,['And you never worry about anything. When we ...,1005
4,jaime lannister,"[""As your brother, I feel it's my duty to warn...",945
5,sansa stark,"['Thank you.', 'Will you shut up?', 'Would you...",784
6,arya stark,"['Move!', ""Where's the Imp?"", 'Arya.', ""That's...",783
7,davos,"['Not now.', ""Stannis is our king. We follow w...",528
8,theon greyjoy,"['Mountain lion?', ""It's a freak."", 'Right. Gi...",455
9,petyr baelish,"[""But we have a Kingdom to look after. I've ho...",449


In [66]:
dataset.iloc[10]['Sentence']

'[\'Jon said he was a deserter.\', \'Our way is the old way?\', \'Is it true he saw the White Walkers?\', \'So he was lying?\', "Where will they go? Their mother\'s dead.", \'NO!\', \'Please, father!\', \'What about you?\', "I saw the king! He\'s got hundreds of people!", "But he\'s coming right now! Down our road!", \'I promise.\', \'What?\', \'Come on, you.\', \'Ten\', \'I hate your stories.\', "Those weren\'t my favorites. My favorites were the scary ones.", \'Maybe we do.\', "I did though. It\'s true, isn\'t it... what Maester Luwin says about my legs? I\'d rather be dead.", "I\'d rather be dead.", "I don\'t want to see anyone.", "I don\'t want to go.", \'Kneel, Hodor.\', \'Yes. Well, I mean I did like to.\', "I\'m not a cripple.", \'Will I really be able to ride?\', \'The Iron Islands. Sigil - a Kraken. Words - "We do not sow".\', \'The Greyjoys.\', "Sigil - a stag. - A crowned stag now that Robert\'s King.", \'Words - "Ours is the fury". Lords - the Baratheons.\', \'The Westerlan

In [59]:
dataset['Sentence_Count'].sum()

14076

In [61]:
dataset.iloc[0]['Sentence']

'[\'Mmh. It is true what they say about the Northern girls.\', \'I did hear something about that.\', \'And the other brother?\', "There\'s the pretty one. And there\'s the clever one.", \'I hear he hates that nickname.\', \'Clever girl.\', \'Have you?\', \'The gods gave me one blessing.\', \'Should I explain to you the meaning of a closed door in a whorehouse, brother?\', \'She has odd cravings, our sister.\', "I\'m sorry, I\'ve begun the feast a bit early. And this is the first of many courses.", \'Close the door!\', "Your uncle\'s in the Night\'s Watch.", "Preparing for a night with your family. I\'ve always wanted to see the Wall.", "My greatest accomplishment. You – you\'re Ned Stark\'s bastard, aren\'t you?", \'Did I offend you? Sorry. You are the bastard, though.\', \'And Lady Stark is not your mother. Making you a bastard. Let me give you some advice, bastard. Never forget what you are. The rest of the world will not. Wear it like armor. Then it can never be used to hurt you.\',

In [80]:
# Run Trait Calculation actual data
character_personality_profiles = {
    dataset.iloc[0]['Name']: calculate_character_traits(dataset.iloc[0]['Sentence'])
}
character_personality_profiles

['bravery', 'valor', 'fearlessness', 'heroism', 'boldness', 'daring']
0.256728857755661
['faithfulness', 'dedication', 'allegiance', 'trustworthiness', 'devotion', 'commitment']
0.21798430383205414
['deception', 'manipulation', 'power play', 'trickery']
0.3038864731788635
['ambition', 'goal-setting', 'career', 'dreams']
0.370542973279953
['sarcasm', 'joke', 'funny', 'laugh', 'hilarious']
0.25190046429634094


{'tyrion lannister': {'courage': 0.256728857755661,
  'loyalty': 0.21798430383205414,
  'manipulativeness': 0.3038864731788635,
  'ambition': 0.370542973279953,
  'humour': 0.25190046429634094}}

In [ ]:
# Run Trait Calculation actual data
character_personality_profiles = {
    record['Name']: calculate_character_traits(record['Name'], record['Sentence'])
    for index,record in dataset.iterrows()
}

# Normalize the scores
normalized_profiles = normalize_traits(character_personality_profiles)


tyrion lannister
0.256728857755661
tyrion lannister
0.21798430383205414
tyrion lannister
0.3038864731788635
tyrion lannister
0.370542973279953
tyrion lannister
0.25190046429634094
jon snow
0.20917578041553497
jon snow
0.22891835868358612
jon snow
0.2986835539340973
jon snow
0.33996862173080444
jon snow
0.25245943665504456
daenerys targaryen
0.22213654220104218
daenerys targaryen
0.22517971694469452
daenerys targaryen
0.3239070773124695
daenerys targaryen
0.31911739706993103
daenerys targaryen
0.2654365003108978
cersei lannister
0.2136654406785965
cersei lannister
0.24063073098659515
cersei lannister
0.29088807106018066
cersei lannister
0.3515455424785614
cersei lannister
0.23998072743415833
jaime lannister
0.22141990065574646
jaime lannister
0.21946880221366882
jaime lannister
0.36735519766807556
jaime lannister


In [ ]:
# Output the results
print("Raw Trait Scores:")
print(character_personality_profiles)
print("\nNormalized Trait Scores:")
print(normalized_profiles)

In [ ]:
# Save to a JSON file
with open("char_scores\got_char_profiles_norm.json", "w") as f:
    json.dump(normalized_profiles, f)

In [ ]:
# Save to a JSON file
with open("char_scores\got_char_profiles_orig.json", "w") as f:
    json.dump(character_personality_profiles, f)